<div class="alert alert-block alert-warning">
<b>Disclaimer:</b> The main objective of this jupyter notebook is to show how to fit simple data by
    
- defining a fitting model
- creating the reference data to which the model will be fitted to. In standard cases, this step should be replaced by loading your experimental data
- setting and running the fit   
- extracting and displaying information about the results

The syntax to fit data is minimizer-dependent. Here we focus on a fitting routing provided by the `scipy` python package: <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html">scipy.optimize.curve_fit</a>.   
    
Please also note that the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.
</div>


<a id='Table of Contents'></a><h1>Table of Contents</h1>

- <a href='#imports'>Importing libraries</a>
- <a href='#def_function'>Definition of the fitting model</a>
- <a href='#anim_plot'>Plot of the fitting model</a>
- <a href='#ref_data'>Creating reference data</a>
- <a href='#fitting'>Setting and fitting</a>
- <a href='#plot'>Displaying the results</a>  
- <a href='#exercise'>Exercises</a>

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Importing libraries</h2>

Please refer to the notebooks about `numpy` and `matplotlib` for more details. 

In [ ]:
# import python modules for plotting, fitting
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# for interactive plot
import ipywidgets

(<a href='#Table of Contents'>Top</a>)<a id='def_function'></a><h2>Definition of the fitting model</h2>

Here we define a simple lorentzian function.  

Please note that there are other options, such as importing [`lmfit.LorentzianModel`](https://lmfit.github.io/lmfit-py/builtin_models.html#lmfit.models.LorentzianModel),  [`QENSmodels.lorentzian`](https://github.com/QENSlibrary/QENSmodels/blob/master/QENSmodels/lorentzian.py) or [`pyspec.fitfuncs.lor`](https://pythonhosted.org/pyspec/fit.html).

In [ ]:
def lorentzian(xx, scale=1.0, center=1.0, hwhm=3.0):
    if hwhm == 0:
        raise ValueError('hwhm of the lorentzian is equal to zero.')
    return scale * hwhm / ((xx-center)**2 + hwhm**2) / np.pi

(<a href='#Table of Contents'>Top</a>)<a id='anim_plot'></a><h2>Plot of the fitting model</h2>

The widget below shows how the variations of the lorentzian's parameters, *Scale*, *Center* and *FWHM*,  influence its shape.

In [ ]:
# define values of the variable of the functions to be used in this tutorial
xx = np.linspace(-10, 10, 500)

initial_params = [5.5, 5.0, 3.0]

fig1, ax1 = plt.subplots()
lines = ax1.plot(xx, lorentzian(xx, *initial_params))
ax1.set_ylabel('lorentzian(x,{},{},{})'.format(*initial_params))
ax1.set_xlabel('x')
ax1.grid()

# define function to be plotted
def interactive_plot(scale, center, hwhm):
    lines[0].set_ydata(lorentzian(xx, scale, center, hwhm))
    plt.ylabel('lorentzian(x,{scale},{center},{hwhm})'.
               format(scale=scale, center=center, hwhm=hwhm))

# define modifiable parameters and their range of variations
interactive_plot = ipywidgets.interactive(interactive_plot, 
                                          scale=(1.0, 10.0), 
                                          center=(0.0, 10.0), 
                                          hwhm=(1.0, 5.0))

# Define function to reset all parameters' values to the initial ones
def reset_values(b):
    """Reset the interactive plots to inital values."""
    for i, p in enumerate(initial_params):
        interactive_plot.children[i].value = p

# Define reset button and occurring action when clicking on it
reset_button = ipywidgets.Button(description = "Reset")
reset_button.on_click(reset_values)

# display the interactive plot
display(interactive_plot, reset_button)

(<a href='#Table of Contents'>Top</a>)<a id='ref_data'></a><h2>Creating reference data</h2>

**Input:** the reference data for this simple example correspond to a Lorentzian with added noise.

In [ ]:
# Creation of reference data
# lorentzian with noise added as background and modulation in its amplitude
added_noise = 0.5*np.random.normal(0, 1, 500)
lorentzian_noisy = lorentzian(xx, 0.89, -0.025, 0.45)*(1. + 0.1*added_noise) + 0.01*added_noise

# Display reference data
fig2, ax2 = plt.subplots()
lines = ax2.plot(xx, lorentzian_noisy, label='reference data')
ax2.set_xlabel('x')
ax2.grid()
ax2.legend()

(<a href='#Table of Contents'>Top</a>)<a id='fitting'></a><h2>Setting and fitting</h2>

The fit is performed using `scipy.optimize.curve_fit`. <br> The example is based on implementations from https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [ ]:
# define set of initial values for the parameters to be refined 
initial_parameters_values = [1, 0.2, 0.5]

# plot initial model and reference data
fig3, ax3 = plt.subplots()
ax3.plot(xx, lorentzian_noisy, label='reference data')
ax3.plot(xx, lorentzian(xx, *initial_parameters_values), '.', label='model with initial guess')
ax3.set_xlabel('x')
ax3.grid()
ax3.legend()

`scipy.optimize_curve_fit` offers the possibility of choosing the method for optimization. Here we are going exploit this option by providing a widget to select your method.

In [ ]:
chosen_method_optim = ipywidgets.RadioButtons(
    options=['lm', 'trf', 'dogbox'],
    value='lm', # Defaults to 'lm'
    description='Method for optimization',
    style={'description_width': 'initial'},
    disabled=False
)

chosen_method_optim

In [ ]:
# perform fit
params, params_covariance = curve_fit(lorentzian, xx, lorentzian_noisy,
                                      method=chosen_method_optim.value,
                                      p0=initial_parameters_values)

(<a href='#Table of Contents'>Top</a>)<a id='plot'></a><h2>Displaying the results</h2>

In [ ]:
# Calculation of the errors on the refined parameters:
params_error = np.sqrt(np.diag(params_covariance))

print('Values of refined parameters:')
print('scale:', params[0],'+/-', params_error[0])
print('center :', params[1],'+/-', params_error[1])
print('HWHM', params[2],'+/-', params_error[2])

In [ ]:
# Comparison of reference data with fitting result
fig4 = plt.figure()
gs = fig4.add_gridspec(3, 1)
f4_ax1 = fig4.add_subplot(gs[0:2, :])
f4_ax2 = fig4.add_subplot(gs[2, :])

f4_ax1.plot(xx, lorentzian_noisy, label='reference data')
f4_ax1.plot(xx, lorentzian(xx, *params), '--', label='fit: %5.3f, %5.3f, %5.3f' % tuple(params))
f4_ax2.plot(xx, lorentzian_noisy - lorentzian(xx, *params), label='residuals')
f4_ax1.set_xlabel('x')
f4_ax1.grid()
f4_ax1.legend()
f4_ax2.set_xlabel('x')
f4_ax2.grid()
f4_ax2.legend()

(<a href='#Table of Contents'>Top</a>)<a id='exercise'></a><h2>Exercise</h2>
This exercise uses the same procedure as above but with slightly different data.  
The objective is to determine the new fitting results.  

**(1)** Determine the initial guesses for the amplitude, center and width, *Scale*, *Center* and *FWHM* to fit the following data based on a Lorentzian profile with added noise by running the cell below and using the generated widget.

In [ ]:
lorentzian_noisy_exo = lorentzian(xx, 3, 4, 0.5) * (
    1. + 0.1*np.random.normal(0,1,500)) + 0.01*np.random.normal(0,1,500)

initial_params = [5.5, 0.0, 0.55]

fig5, ax5 = plt.subplots()
ax5.plot(xx, lorentzian_noisy_exo, label="reference data for exercise")
lines = ax5.plot(xx, lorentzian(xx, *initial_params), label='model to be fitted')
ax5.set_ylabel('lorentzian(x,{},{},{})'.format(*initial_params))
ax5.set_xlabel('x')
ax5.grid()
ax5.legend()

def interactive_plot_exo(scale, center, hwhm):
    lines[0].set_ydata(lorentzian(xx, scale, center, hwhm))
    plt.ylabel('lorentzian(x,{scale},{center},{hwhm})'.
               format(scale=scale, center=center, hwhm=hwhm))

interactive_plot_exo = ipywidgets.interactive(interactive_plot_exo, 
                                          scale=(1.0, 10.0), 
                                          center=(-5.0, 5.0), 
                                          hwhm=(0.1, 1.0))

# Define function to reset all parameters' values to the initial ones
def reset_values(b):
    """Reset the interactive plots to inital values."""
    for i, p in enumerate(initial_params):
        interactive_plot_exo.children[i].value = p

# Define reset button and occurring action when clicking on it
reset_button_exo = ipywidgets.Button(description = "Reset")
reset_button_exo.on_click(reset_values)

# display the interactive plot
display(interactive_plot_exo, reset_button_exo)

**(2)** Running the following cells will:
- assign the values you determined just above to your model
- plot the initial model and the experimental data
- run the fit

In [ ]:
initial_parameters_values_exo = [interactive_plot_exo.children[i].value for i in range(3)]
initial_parameters_values_exo

The syntax above uses list comprehensions (as described in another tutorial). Options for such inputs are:
- list comprehension
```python
x = [i for i in range(10)]```
- with a standard `for` loop:
```python
x = []
for i in range(10):
    x[i] = i
```
- entered by hand
```python 
x = [0, 1, 2, 3, , 4, 5, 6, 7, 8, 9]```

In [ ]:
fig6, ax6 = plt.subplots()
ax6.plot(xx, lorentzian_noisy_exo, label="reference data for exercise")
ax6.plot(xx, lorentzian(xx, *initial_parameters_values_exo), '.', label='model with initial guesses')
ax6.set_ylabel('lorentzian(x,{},{},{})'.format(*initial_params))
ax6.set_title("Exercise: plot before fit")
ax6.set_xlabel('x')
ax6.grid()
ax6.legend()

Now run the fit

In [ ]:
params_exo, pcov_exo = curve_fit(lorentzian, xx, lorentzian_noisy_exo, 
                               p0=initial_parameters_values_exo)

**(3)** Display the final values of the refined parameters

In [ ]:
print('Values of refined parameters:')
print('scale:', params_exo[0])
print('center :', params_exo[1])
print('HWHM', params_exo[2])
print('to be compared to 3, 4, 0.5')

**(4)** Plot the reference data and the model  

In the code below, replace:    
 - `{A}` with name of reference data  
 - `{B}` with name of list of values of fitted parameters 

Refer to the <a href='#plot'>first example</a> for help

In [ ]:
fig7 = plt.figure()
gs = fig7.add_gridspec(3, 1)
f7_ax1 = fig7.add_subplot(gs[0:2, :])
f7_ax2 = fig7.add_subplot(gs[2, :])

f7_ax1.plot(xx, {A}, label='reference data for exercise')
f7_ax1.plot(xx, lorentzian(xx, *{B}), '--', label='fit: %5.3f, %5.3f, %5.3f' % tuple({B}))
f7_ax2.plot(xx, {A} - lorentzian(xx, *{B}), label='residuals')
f7_ax1.set_xlabel('x')
f7_ax1.grid()
f7_ax1.legend()
f7_ax2.set_xlabel('x')
f7_ax2.grid()
f7_ax2.legend()

**(5) Bonus: putting it all together!**

Try to gather everything we've done above under a single interface:
- an interactive plot that shows raw-data, initial guess and fit residuals
- a `RadioButton` widget to choose fit optimization method
- a `Button` which, when clicked, runs the fit and updates the fitted curve and residuals in the figure

# Additional references
Other fitting packages could be used. For example,
- [bumps](https://bumps.readthedocs.io/en/latest/)
- [lmfit](https://lmfit.github.io/lmfit-py/)

For further details about `ipywidgets`, please refer to the following [link](https://ipywidgets.readthedocs.io/en/latest/index.html).